# Unsupervised Learning and Probabilistic Models

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

TensorFlow 1.x selected.


In [ ]:
def reduce_logsumexp(input_tensor, reduction_indices=1, keep_dims=False):
  """Computes the sum of elements across dimensions of a tensor in log domain.

     It uses a similar API to tf.reduce_sum.

  Args:
    input_tensor: The tensor to reduce. Should have numeric type.
    reduction_indices: The dimensions to reduce. 
    keep_dims: If true, retains reduced dimensions with length 1.
  Returns:
    The reduced tensor.
  """
  max_input_tensor1 = tf.reduce_max(
      input_tensor, reduction_indices, keep_dims=keep_dims)
  max_input_tensor2 = max_input_tensor1
  if not keep_dims:
    max_input_tensor2 = tf.expand_dims(max_input_tensor2, reduction_indices)
  return tf.log(
      tf.reduce_sum(
          tf.exp(input_tensor - max_input_tensor2),
          reduction_indices,
          keep_dims=keep_dims)) + max_input_tensor1


def logsoftmax(input_tensor):
  """Computes normal softmax nonlinearity in log domain.

     It can be used to normalize log probability.
     The softmax is always computed along the second dimension of the input Tensor.     

  Args:
    input_tensor: Unnormalized log probability.
  Returns:
    normalized log probability.
  """
  return input_tensor - reduce_logsumexp(input_tensor, reduction_indices=0, keep_dims=True)

#K-means


## Learning K-means



In [ ]:
# Distance function for K-means
def distanceFunc(X, MU):
    # Inputs
    # X: is an NxD matrix (N observations and D dimensions)
    # MU: is an KxD matrix (K means and D dimensions)
    # Outputs
    # pair_dist: is the pairwise distance matrix (NxK)

    X_expand = tf.expand_dims(X, 0)
    MU_expand = tf.expand_dims(MU, 1)

    sqr_distance = tf.square(tf.subtract(X_expand, MU_expand))
    sqr_distance = tf.reduce_sum(sqr_distance, axis = 2)
    sqr_distance = tf.transpose(sqr_distance)

    return sqr_distance

In [ ]:
def buildGraphKM(data, data_dim, K, lr):

  data_points = tf.placeholder(tf.float32, (None, data_dim), name = "data")

  # Initialize the cluster centers based on a random sample of
  # data points in the set...
  centers = tf.Variable(tf.cast(tf.slice(tf.random_shuffle(data), [0, 0], (K, -1)), dtype=tf.float32), dtype=tf.float32)

  # Out loss function is the pairwise distance between points and 
  # their cluster centers...
  loss = tf.reduce_sum(tf.reduce_min(distanceFunc(data_points, centers), axis = 1))

  optim = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.9, beta2=0.99, epsilon=1e-5)
  optim = optim.minimize(loss)

  return data_points, centers, optim, loss

In [ ]:
def plotScatter(title, sample_points, centers, K, size, assign, min_val_loss):
  frequency = np.bincount(assign)
  index = np.nonzero(frequency)[0]
  frequency = zip(index, frequency[index])

  iter = 1
  percentages = []
  for i in frequency:
    percentages.append("Cluster " + str(i[0] + 1) + " " + str(round((i[1] / len(assign)) * 100, 2)) +  "%")
    iter += 1

  plt.title(title)
  
  for i in range(K):
    plt.scatter(sample_points[i][:, 0], sample_points[i][:, 1], alpha=1, s=5, label = percentages[i])

  plt.scatter(centers[:, 0], centers[:, 1], marker='x', s = 50, c='black')

  plt.text(0, -6.5, "Validation loss: " + str(min_val_loss), ha='center')

  plt.legend()
  plt.show()

In [ ]:
def sampleColours(sample_points, centers):
  # For each sample find the closest center and then assign it a colour...
  closest = tf.arg_min(distanceFunc(sample_points, centers), 1)
  return closest

In [ ]:
def K_means(K, lr, is_valid = False, epochs=10, plot=True, npy=2):
  train_loss = []
  val_loss = []

  # Loading data
  if (npy == 2):
    data = np.load('data2D.npy')
  else:
    data = np.load('data100D.npy')
  [num_pts, dim] = np.shape(data)

  # For Validation set
  if is_valid:
    valid_batch = int(num_pts / 3.0)
    np.random.seed(45689)
    rnd_idx = np.arange(num_pts)
    np.random.shuffle(rnd_idx)
    val_data = data[rnd_idx[:valid_batch]]
    data = data[rnd_idx[valid_batch:]]
  
  X, MU, optim, loss = buildGraphKM(data, dim, K, lr)

  init_op = tf.global_variables_initializer()
  tf.set_random_seed(1000)

  with tf.Session() as sess:
    sess.run(init_op)
    print("Starting K-means...")

    feed = {
      X : data
    }

    for itteration in range(epochs):
      centers, losses, _ = sess.run([MU, loss, optim], feed_dict = feed)

      train_loss.append(losses / len(data))

      # Calculate the validation loss
      if is_valid:
        valid_center, valid_loss, _ = sess.run([MU, loss, optim], feed_dict  = {X : val_data})
        val_loss.append(valid_loss / len(val_data))


    colours = sess.run(sampleColours(data, centers), feed_dict = {X: data, MU : centers})
  
  cluster_data = []

  for i in range(K):
    cluster_data.append(data[colours == i])

  if (plot):
    min_loss = round(val_loss[np.argmin(val_loss)], 4)

    plotScatter("K-means Clustering", cluster_data, centers, K, len(data), colours, min_loss)

  plt.title("K-means loss")
  plt.plot(train_loss, label="Training loss")
  plt.plot(val_loss, label="Validation loss")
  plt.legend()
  plt.show()
  return train_loss, val_loss

In [ ]:
num_clusters = [1, 2, 3, 4, 5]

for k in num_clusters:
  train_loss, val_loss = K_means(k, 0.1, is_valid=True, epochs=150)

# Mixture of Gaussians

## The Gaussian cluster model


$
P(x | z) = \frac{P(z | x)P(x)}{P(z)}
$

In [ ]:
# Distance function for GMM
def distanceFunc(X, MU):
    # Inputs
    # X: is an NxD matrix (N observations and D dimensions)
    # MU: is an KxD matrix (K means and D dimensions)
    # Outputs
    # pair_dist: is the pairwise distance matrix (NxK)

    X_expand = tf.expand_dims(X, 0)
    MU_expand = tf.expand_dims(MU, 1)

    sqr_distance = tf.square(tf.subtract(X_expand, MU_expand))
    sqr_distance = tf.reduce_sum(sqr_distance, axis = 2)
    sqr_distance = tf.transpose(sqr_distance)

    return sqr_distance

def log_GaussPDF(X, mu, sigma):
    # Inputs
    # X: N X D
    # mu: K X D
    # sigma: K X 1

    # Outputs:
    # log Gaussian PDF N X K

    # Recall pdf of gaussian:
    # pdf(x; mu, sigma) = exp(-0.5 (x - mu)**2 / sigma**2) / Z
    # Z = (2 pi sigma**2)**0.5
    # log(pdf) = (-0.5 (x - mu)**2 / sigma ** 2) - log(Z)

    data_dim = tf.cast(X.shape[1], tf.float32)
    Z = (2 * np.pi * tf.square(sigma))
    Z = (data_dim/2) * tf.log(Z)
    pdf = -0.5 * tf.divide(distanceFunc(X, mu), tf.squeeze(tf.square(sigma)))

    log_gauss = pdf - tf.transpose(Z)
    return log_gauss 

def log_posterior(log_PDF, log_pi):
    # Input
    # log_PDF: log Gaussian PDF N X K
    # log_pi: K X 1

    # Outputs
    # log_post: N X K

    # log[P(z | x)] = log[P(x | z)] + log[P(z)] - 
    #                 log[Sum_1_N{exp(log[P(x | z)] + log[P(z)])}] 

    log_post_num = tf.add(log_PDF, tf.transpose(log_pi)) 
    log_post_dem = reduce_logsumexp(log_PDF + log_pi, keep_dims=True)

    return log_post_num - log_post_dem

## Learning the MoG

In [ ]:
def buildGraphGMM(data, data_dim, K, lr, div):

  data_points = tf.placeholder(tf.float32, (None, data_dim), name = "data")

  # Initialize the cluster centers based on a random sample of
  # data points in the set..
  centers = tf.Variable(tf.cast(tf.slice(tf.random_shuffle(data), [0, 0], (K, -1)), dtype=tf.float32), dtype=tf.float32) 

  # Initialize the standard deviation of each function to be a random
  # sample from a gaussian distribution
  # Make it exponential to deal with constraint...
  sigma = tf.Variable(tf.random_normal((K, 1), stddev= div), dtype=tf.float32)
  sigma = tf.exp(sigma)

  # The weights of each distribution 
  # But again because we're using exp sigma, we need to change 
  # pi_k to be represented by a softmax function
  pi = tf.Variable(tf.random_normal((K, 1), stddev= div), dtype=tf.float32)
  pi = tf.squeeze(logsoftmax(pi))

  # Loss = -log[P(X)]
  #      = -log[π_1_N{P(x_n)}]
  #      = -log[π_1_N{Σ_1_K{π_k N(x_n)}}]
  #      = -log[π_1_N{Σ_1_K{e ^ (log[π_k] + log[N(x_n)])}}]
  #      = -log[Σ_1_K{e ^ (log[π_k] + log[N(x_1)])}] - ... 
  #        -log[Σ_1_K{e ^ (log[π_k] + log[N(x_N)])}]
  log_pdf = log_GaussPDF(data_points, centers, sigma)
  loss = reduce_logsumexp(log_pdf + pi, 1, keep_dims=True)
  loss = -1 * tf.reduce_sum(loss)

  optim = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.9, beta2=0.99, epsilon=1e-5)
  optim = optim.minimize(loss)

  # Based on the distributions make a prediction as to which 
  # cluster a data point belongs too
  classify = log_posterior(log_pdf, pi)
  classify = tf.nn.softmax(classify)
  classify = tf.argmax(classify, axis = 1)

  return data_points, centers, sigma, pi, optim, loss, classify

In [ ]:
def GMM(K, lr, stddve, is_valid = False, epochs=10, plot=True, npy=1):
  train_loss = []
  val_loss = []
  train_cluster = []

  # Loading data
  if (npy == 2):
    data = np.load('data2D.npy')
  else:
    data = np.load('data100D.npy')
  
  [num_pts, dim] = np.shape(data)

  # For Validation set
  if is_valid:
    valid_batch = int(num_pts / 3.0)
    np.random.seed(45689)
    rnd_idx = np.arange(num_pts)
    np.random.shuffle(rnd_idx)
    val_data = data[rnd_idx[:valid_batch]]
    data = data[rnd_idx[valid_batch:]]
  
  X, MU, sigma, pi, optim, loss, classify = buildGraphGMM(data, dim, K, lr, stddve)

  init_op = tf.global_variables_initializer()
  tf.set_random_seed(1000)

  with tf.Session() as sess:
    sess.run(init_op)
    print("Starting GMM...")

    feed = {
      X : data
    }

    for itteration in range(epochs):
      centers, losses, cluster_assign, _ = sess.run([MU, loss, classify, optim], feed_dict = feed)

      train_loss.append(losses / len(data))
      train_cluster.append(cluster_assign)

      if is_valid:
        valid_center, valid_loss, _c, _o = sess.run([MU, loss, classify, optim], feed_dict = {X : val_data})
        val_loss.append(valid_loss / len(val_data))
    
    # Create scatter plot
    cluster_data = []
    
    for i in range(K):
      cluster_data.append(data[cluster_assign == i])

    # print("Trained parameters:", sess.run(sigma), sess.run(pi), centers)

  if (plot):  
    min_loss = round(val_loss[np.argmin(val_loss)], 4)

    plotScatter("GMM Clustering", cluster_data, centers, K, len(data), cluster_assign, min_loss)

  plt.title("GMM loss")
  plt.plot(train_loss, label="Training loss")
  plt.plot(val_loss, label="Validation loss")
  plt.legend()
  plt.show()

  return train_loss, val_loss

Best learned parameters with K = 3 <br>
$\sigma :$ 0.9934453 ,0.1977245 <br>
$\pi :$ -1.094792, -1.1026771, -1.0983831 <br>
$\mu :$ [ 0.10631254, -1.5272143 ], [-1.1020751,  -3.306512  ], [ 1.2982769,   0.30938783] <br>

In [ ]:
train_loss, val_loss = GMM(3, 0.01, 0.05, True, 100)

In [ ]:
num_clusters = [1, 2, 3, 4, 5]

for k in num_clusters:
  train_loss, val_loss = GMM(k, 0.01, 0.05, True, 500)

In [ ]:
num_clusters = [5, 10, 15, 20, 30]

for k in num_clusters:
  train_loss, val_loss = GMM(k, 0.01, 0.05, True, 200, plot=False, npy=100)
  print("Training loss:", train_loss[np.argmin(train_loss)])
  print("Validation loss:", val_loss[np.argmin(val_loss)])

In [ ]:
loss_over_time = [85.4696407140714, 48.51731266876688, 48.28689431443144, 47.97822438493849, 47.87642983048305]
K = [5, 10, 15, 20, 30]

plt.plot(K, loss_over_time)
plt.show()

In [ ]:
num_clusters = [5, 10, 15, 20, 30]

for k in num_clusters:
  train_loss, val_loss = K_means(k, 0.01, True, 200, plot=False, npy=100)
  print("Training loss:", train_loss[np.argmin(train_loss)])
  print("Validation loss:", val_loss[np.argmin(val_loss)])

In [ ]:
loss_over_time = [21.659822232223224, 20.872030171767175, 20.534962871287128, 20.391445394539453, 20.010754200420042]
K = [5, 10, 15, 20, 30]

plt.plot(K, loss_over_time)
plt.show()